In [9]:
import collections, re
import nltk
from nltk.corpus import stopwords
import os
import string
import numpy as np
import pandas as pd
from collections import defaultdict
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag import StanfordNERTagger
import pickle

In [21]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ericdong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ericdong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ericdong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ericdong/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/ericdong/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/ericdong/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [11]:
with open('cleaned-data.pkl', 'rb') as handle:
    recipes = pickle.load(handle)

In [35]:
recipes['pasta al pesto']

{'Categories': ['Main dish', 'Pasta', 'Vegetables'],
 'Yield': '6',
 'Ingredients': ['8 oz Pasta (preferably linguine)',
  '3 x  Carrots, thinly sliced',
  '2 tb Safflower or Olive oil',
  '3 x  Sm Zucchini, thinly sliced',
  '1/4 lb Peapods',
  'MMMMM---------------------------PESTO--------------------------------',
  '2 c  Fresh Basil Leaves',
  '1/4 c  Pine nuts (pignolli)',
  '2 x  Cloves Garlic',
  '1 tb Olive oil'],
 'Instructions': "PASTA GARNISH: freshly ground black pepper and Parmesan cheese,  optional '''''''''''''''''''''''''''''''''''''''''''''''''''''''  '''''''''''''''' PESTO: Place ingredients in bowl of food processor.  Process until smooth, using rubber scraper to push down the sides  occasionally. Makes 1/2 cup. VARIATIONS: - add 3/4 c freshly grated  Parmesan Cheese - subst. cream cheese, kefir, or Neufchatel cheese  for oil - subst. walnuts or hazelnuts for pine nuts PASTA: Boil a  large pot of water; cook pasta until al dente. While pasta is  cooking, prepare pest

In [36]:
food_words = ['food', 'cook', 'fruit', 'vegetable', 'meat', 'eat', 'taste', 'fat', 'dairy', 'grain', 
              'drink', 'beverage', 'alcohol', 'bread', 'flavor', 'spice', 'dessert']
food_synsets = set([synset for food in food_words for synset in wordnet.synsets(food)])
diff = len(food_synsets)

while diff > 0:
    hyponyms = set([hyponym for synset in food_synsets for hyponym in synset.hyponyms()])
    old_len = len(food_synsets)
    food_synsets = food_synsets.union(hyponyms)
    new_len = len(food_synsets)
    diff = new_len - old_len

print(len(food_synsets))

2976


In [37]:
lemmatizer = WordNetLemmatizer()

In [48]:
relevant_entities = ['ORGANIZATION', 'PERSON', 'LOCATION']

def has_ne(instruction, relevant_entities):
    instruction_tree = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(instruction)))
    for subtree in instruction_tree.subtrees():
        if subtree.label() in relevant_entities:
            return True
    
    return False

In [50]:
count = 0
for title, recipe in recipes.items():
    if 'Categories' in recipe:
        recipes[title]['Categories'] = [category.lower() for category in recipe['Categories']]
    if 'Ingredients' in recipe:
        ingredients = nltk.pos_tag([lemmatizer.lemmatize(token.lower()) for ingredient in recipe['Ingredients'] 
                                    for token in nltk.word_tokenize(ingredient)])
        ingredients = [word for word, pos in ingredients if pos == 'NN' and 
                       not set(wordnet.synsets(word)).isdisjoint(food_synsets)]
        recipes[title]['Ingredients'] = ingredients
    if 'Instructions' in recipe:
        instructions = nltk.sent_tokenize(recipe['Instructions'])
        cleaned_instructions = [instruction.lower() for instruction in instructions 
                                if not has_ne(instruction, relevant_entities)]
        recipes[title]['Instructions'] = cleaned_instructions
    count += 1
    if (count % 4500):
        print('Processed %d recipes' % count)

In [61]:
recipes['cauliflower and red lentil curry']

{'Categories': ['main dish', 'indian', 'vegetarian', 'curries'],
 'Yield': '4',
 'Ingredients': ['lentil',
  'oil',
  'onion',
  'cumin',
  'seed',
  'clove',
  'garlic',
  'ginger',
  'cayenne',
  'plum',
  'tomato',
  'lemon',
  'juice',
  'c',
  'cauliflower',
  'cilantro',
  'jalapeno',
  'pepper',
  'sugar'],
 'Instructions': ['in a large saucepan over low heat, combine lentils, onions, curry  powder, salt, turmeric, and 2 cups water;  bring to a simmer.',
  'cover  and cook, stirring occasionally, until the lentils are soft and the  sauce has thickened, about 45 minutes.',
  'add tomatoes, cauliflower,  and jalapeno peppers and simmer, covered, until the cauliflower is  tender, 8 to 10 minutes longer.',
  'remove from heat.',
  'heat oil in a small skillet over medium-high heat.',
  'add cumin seeds  and cook for about 10 seconds.',
  'add garlic and ginger;  saute until  the garlic is lightly browned, about 1 minute.',
  'stir in cayenne and  immediately add the oil-spice mixtur

In [52]:
len(recipes)

96361

In [54]:
file = open('processed-data.pkl', 'wb')
pickle.dump(recipes, file)
file.close()